In [ ]:
### Naver Movie
- robots.txt 설정으로 크롤링 안됩니다. settings.py에 robots.txt 설정 바꾸면 크롤링 됨.

### 1. 프로젝트 생성

In [1]:
!scrapy startproject naver_movie

New Scrapy project 'naver_movie', using template directory 'c:\users\user\appdata\local\continuum\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Code\04 Web\scrapy\naver_movie

You can start your first spider with:
    cd naver_movie
    scrapy genspider example example.com


### 2. items 설정
- https://movie.naver.com/movie/running/current.nhn
- 제목, 관객수, 평점

In [2]:
!cat naver_movie/naver_movie/items.py
# 여기 나오는 설명 참고해서 아래와 같이 class 작성

# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class NaverMovieItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [4]:
%%writefile naver_movie/naver_movie/items.py
import scrapy

class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()

Overwriting naver_movie/naver_movie/items.py


### 3. xpath 확인

In [8]:
import requests
import scrapy
from scrapy.http import TextResponse

In [9]:
# 링크
req = requests.get("https://movie.naver.com/movie/running/current.nhn")
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [13]:
links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
len(links), links[0]

(74, '/movie/bi/mi/basic.nhn?code=189001')

In [15]:
link = response.urljoin(links[0]) #url로 만들어주기(도메인 추가)
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189001'

In [16]:
# 상세 데이터 수집
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [18]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0] #리스로 가져오기때문에 [0]붙여서 문자로
count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()[0]
star = "".join(star)

title, count, star

('인비저블맨', '153,784명', '9')

### 4. spider 작성

In [34]:
%%writefile naver_movie/naver_movie/spiders/spider.py
import scrapy
from naver_movie.items import NaverMovieItem


class MovieSpider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/running/current.nhn"]

    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_page_contents)

    def parse_page_contents(self, response):
        item = NaverMovieItem()
        item["title"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item["count"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
        except:
            item["count"] = "0명"
        star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item["star"] = "".join(star)
        yield item

Overwriting naver_movie/naver_movie/spiders/spider.py


### 5.scrapy 실행

In [35]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovie -o naver_movie.csv

Overwriting run.sh


In [36]:
!chmod +x run.sh

In [38]:
!run.sh

### 6. settings.py 파일 변경
- Forbidden by robots.txt

In [27]:
!head -n 25 naver_movie/naver_movie/settings.py | tail -n 5

# Obey robots.txt rules
ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32


In [29]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_movie/naver_movie/settings.py
#문자열 치환해주는 코멘드

In [30]:
!head -n 25 naver_movie/naver_movie/settings.py | tail -n 5

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32


In [31]:
!run.sh

In [39]:
import pandas as pd

In [41]:
df = pd.read_csv("naver_movie/naver_movie.csv")
df.head(2)

,count,star,title
0,"153,784명",9.24,인비저블맨
1,"5,558명",9.31,시원찮은 그녀를 위한 육성방법 피날레
